## Rice University

This script serves as a basic tutorial for extracting courses of interest from a university. This is by no means the only (or even best way) to go about this process—so if you come up with a process that works better, feel free to implement! If you're unfamiliar with any of the libraries, the comments below annotate reasoning behind each.

In [184]:
import sys
import pandas as pd
import numpy as np
import time
import re
import urllib.request #handles urls
from urllib.request import urlopen
import urllib.parse 
import linkGrabber #extracts urls
import json #encodes/decodes json 
import csv 
import requests #downloads a webpage to scrape
from bs4 import BeautifulSoup, NavigableString, Tag #beautifulsoup pulls data from HTML
import nltk #NLP tasks
from nltk import word_tokenize
from nltk.stem import PorterStemmer #removes word endings
stemmer = PorterStemmer()

The first thing we want to do is set up a function for standard preprocessing. It's also useful to list all of the URLs we'll need to send requests to before scraping. We want all courses within a 2 year *academic* calendar (as opposed to an annual calendar). 

In [185]:
#keyword preprocessing
def preprocess(keyword):
    keyword = keyword.lower() #lowercase
    keyword = word_tokenize(keyword) #tokenize
    for word in keyword:
        keyword = stemmer.stem(word) #stem 
    return (keyword)

Next, we'll want to import our keyword csv, split our keyword lists, and preprocess them. The way the csv is set up, we'll want to split the words that are indicated as technical (`T`) or normative (`N`) and that we've chosen to include (`Y`). You'll notice that preprocessing is useful for some of our words but not for others. Here, we've chosen to manually alter words that are not usefully preprocessed. In this case, it means replacing instances of words that are stemmed to end in i.

[regex is a bitch here]

In [186]:
#import keywords
keywords = pd.read_csv("../keywords.csv")
technical = keywords[(keywords['Technical/Normative']=='T') & (keywords['Include']=='Y')].Keyword
normative = keywords[(keywords['Technical/Normative']=='N') & (keywords['Include']=='Y')].Keyword
normative = [preprocess(i) for i in normative]
technical = [preprocess(i) for i in technical] 

#replace keywords of interest
normative = [w.replace('privaci', 'privac') for w in normative]
normative = [w.replace('democraci', 'democra') for w in normative]
normative = [w.replace('equiti', 'equit') for w in normative]
normative = [w.replace('histori', 'histor') for w in normative]
normative = [w.replace('justice', 'justic') for w in normative]
normative = [w.replace('liberti', 'libert') for w in normative]
normative = [w.replace('philosophi', 'philosoph') for w in normative]
normative = [w.replace('societi', 'societ') for w in normative]
normative = [w.replace('polici', 'polic') for w in normative]

technical = [w.replace('ai', '^ai') for w in technical]
technical = [w.replace('cs', '^cs') for w in technical]
technical = [w.replace('ict', '^ict') for w in technical]
technical = [w.replace('ml', '^ml') for w in technical]
technical = [w.replace('nlp', '^nlp') for w in technical]

print(normative)
print(technical)

['account', 'critic', 'democra', 'discrimin', 'equal', 'equit', 'ethic', 'fair', 'femin', 'gender', 'govern', 'histor', 'inequ', 'justic', 'law', 'legal', 'libert', 'moral', 'norm', 'philosoph', 'polit', 'power', 'privac', 'race', 'religi', 'respons', 'right', 'secur', 'social', 'societ', 'surveil', 'transpar', 'valu', 'polic']
['^ai', 'algorithm', 'analyt', 'intellig', 'automat', 'code', 'comput', '^cs', 'cyber', 'data', 'digit', '^ict', 'inform', 'intelligen', 'internet', 'machin', '^ml', 'process', '^nlp', 'platform', 'program', 'robot', 'softwar', 'system', 'technolog']


The process behind extracting relevant courses works in two steps:
1. First, we want to find and extract all courses that contain any instance of a normative keyword.
2. Then, we want search within these courses to see if it also contains a technical keyword.

We initialize a data frame with columns for all of the course items we want to extract. It probably makes the most sense to standardize these feature names across all university scripts so that they're easier to merge in the final compiled dataset for all universities. Our items of interest are:
* The course title: `title`
* The department and course number: `dept_num`
* The course description: `description`
* The number of credits for the course: `credits`
* The course instructor: `instructor`
* The link to the course syllabus (if applicable): `syllabus`
* The university the course is extracted from: `university`
* The term that the course is offered during (fall, spring, summer / year): `term`
* The keyword that triggered the extraction (this is for auditing purposes): `keyword`

In [187]:
#init dfs
rice_list = []

In [188]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
driver = webdriver.Chrome()

#course catalog URLs - 2 academic years 
terms = ['Fall 2017', 'Spring 2018', 'Summer 2018', 'Fall 2018', 'Spring 2019', 'Summer 2019']

for term in terms:
    print('--------------------')
    print('--------------------')
    print(term)
    
    for word in normative:
        driver.get("https://courses.rice.edu/courses/swkscat.main")
        time.sleep(2)
        
        select = Select(driver.find_element_by_xpath('//*[@id="term_input_id"]'))
        submit = driver.find_element_by_xpath('//*[@id="id____UID0"]')
        select.select_by_visible_text(term)
        submit.click()
        time.sleep(2)

        # for word in normative:
        subject_field = driver.find_element_by_xpath('//*[@id="subj_id"]') 
        subject_select = Select(subject_field)

        #select all subjects
        for subject in subject_field.find_elements_by_tag_name('option'):
            subject_select.select_by_visible_text(subject.text)

        text_input = driver.find_element_by_xpath('//*[@id="title_id"]')
        
        print('--------------------')
        print(word)
        
        text_input.send_keys(word)
        get_course = driver.find_element_by_xpath('//*[@id="id____UID0"]')
        get_course.click()
        time.sleep(2)

        all_courses = driver.find_element_by_xpath('//*[@id="contentHolder"]/div[2]/table[1]/tbody')
        courses = all_courses.find_elements_by_tag_name('td')

        full_titles = courses[0::2]
        descriptions = courses[1::2]
        
        #Has result
        if (len(courses) > 2):
            counter = 0

            for full_title, description in zip(full_titles, descriptions):
                print(counter)
                counter += 1

                georgetown_dict = {}
                title_split = full_title.text.split('-')
                dept_num = title_split[0]
                georgetown_dict['dept_num'] = dept_num
                
                credit_regex = r'[0-9]\.[0-9]{3} Credit hours'
                credits = re.findall(credit_regex, description.text)[0].replace('Credit hours', '')
                georgetown_dict['credits'] = credits

                title = title_split[1]
                georgetown_dict['title'] = title
                georgetown_dict['description'] = description.text
                georgetown_dict['term'] = term
                georgetown_dict['keyword'] = word
                georgetown_list.append(georgetown_dict)

driver.close()

--------------------
--------------------
Fall 2017
--------------------
account
--------------------
critic
--------------------
democra
--------------------
discrimin
--------------------
equal
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
--------------------
equit
--------------------
ethic
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
--------------------
fair
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
--------------------
femin
--------------------
gender
--------------------
govern
--------------------
histor
--------------------
inequ
--------------------
justic
--------------------
law
--------------------
legal
0
1
--------------------
libert
--------------------
moral
--------------------
norm
0
1
2
3
--------------------
philosoph
--------------------
polit
0
1
2
3
4
5
6
7
8
--------------------
power
0
1
--------------------
privac
--------------------
race
0
1
2
------------

--------------------
secur
0
1
2
3
4
5
6
7
8
9
10
11
12
13
--------------------
social
--------------------
societ
0
1
--------------------
surveil
--------------------
transpar
--------------------
valu
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
--------------------
polic


In [189]:
georgetown = pd.DataFrame(georgetown_list)
georgetown

,credits,dept_num,description,keyword,term,title
0,3.000,BLHS 485,Why do some people have more of the good thing...,equal,Fall 2017,Social Inequality and Poverty
1,3.000,DEVM 644,3.000 Credit hours\n3.000 Lecture hours\n\nLev...,equal,Fall 2017,Social Inequality
2,3.000,ECIL 284,3.000 Credit hours\n3.000 Lecture hours\n\nLev...,equal,Fall 2017,Poverty and Inequality
3,3.000,ECIL 684,3.000 Credit hours\n3.000 Lecture hours\n\nLev...,equal,Fall 2017,Poverty and Inequality
4,3.000,ECON 256,This class will explore inequality within and ...,equal,Fall 2017,"Poverty, Growth & Inequality"
5,3.000,ECON 621,Concerns about “poverty” and “inequality” have...,equal,Fall 2017,Econ of Poverty/Inequality
6,3.000,GOVT 240,"Inequality – economic, social, and political –...",equal,Fall 2017,Politics of Inequality
7,3.000,IPEC 328,This course examines the politics of income di...,equal,Fall 2017,Pol Econ Inequality & Redistri
8,3.000,LAWJ 1335,0.000 TO 3.000 Credit hours\n0.000 TO 3.000 Le...,equal,Fall 2017,"Race, Inequality & Just Sem"
9,3.000,LSHV 447,3.000 Credit hours\n3.000 Lecture hours\n0.000...,equal,Fall 2017,Social Inequality


The loop below executes part 1 of our extraction. It's long and kind of messy (sorry), so feel free to play around with the structure if you'd like. The key tasks here are to extract our items of interest based on our search queries and append them to our data frame.

Now that we've extracted all courses containing a normative keyword of interest, we need to filter our courses to only return titles that contain a normative AND a technical keyword. This is the case for all words except instances of our preprocessed `privac` and `secur`, for which we want to return all courses, even if they don't contain two keywords. To do this, we'll split the courses into two data frames, apply our respective conditions, and then merge them back together. 

In [190]:
exceptions = georgetown.loc[(georgetown['keyword']=='privac') | (georgetown['keyword'] =='secur')]
exceptions

,credits,dept_num,description,keyword,term,title
124,1.000,COSC 437,1.000 Credit hours\n1.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybersecurity Seminar
125,1.000,COSC 438,1.000 Credit hours\n1.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybersecurity Writing Seminar
126,2.000,LAWG 1409,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybercrime & Tech Insecurity
127,2.000,LAWG 2004,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Biosecurity and the Law
128,2.000,LAWG 2052,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybersecurity Law
129,2.000,LAWG 3038,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Biosecurity and the Law
130,2.000,LAWJ 1409,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybercrime & Tech Insecurity
131,2.000,LAWJ 2004,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Biosecurity and the Law
132,2.000,LAWJ 3038,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Biosecurity and the Law
133,3.000,MSFS 556,Cybersecurity has emerged as one of the most s...,secur,Fall 2017,Cybersecurity: Bus/Gov/Int Com


In [191]:
#loop through technical keyword list, extract relevant titles
for word in technical:
    df = georgetown[georgetown['title'].str.contains(word, flags = re.IGNORECASE)]
    df['keyword2'] = word
    
#join keyword cols
df["keyword"] = df["keyword"].map(str) + "," + df["keyword2"]
df = df.drop(columns="keyword2")

df

/Users/Shiying/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,credits,dept_num,description,keyword,term,title


NOTE: the above cell is likely not the best nor most simple way to execute this step! Feel free to take special liberties here. It's probably wise to pick out a few titles that you know should be returned manually, then check to see if the script is working as desired. 

In [192]:
#combine dfs 
georgetown = pd.concat([df, exceptions])
georgetown

,credits,dept_num,description,keyword,term,title
124,1.000,COSC 437,1.000 Credit hours\n1.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybersecurity Seminar
125,1.000,COSC 438,1.000 Credit hours\n1.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybersecurity Writing Seminar
126,2.000,LAWG 1409,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybercrime & Tech Insecurity
127,2.000,LAWG 2004,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Biosecurity and the Law
128,2.000,LAWG 2052,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybersecurity Law
129,2.000,LAWG 3038,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Biosecurity and the Law
130,2.000,LAWJ 1409,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Cybercrime & Tech Insecurity
131,2.000,LAWJ 2004,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Biosecurity and the Law
132,2.000,LAWJ 3038,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,secur,Fall 2017,Biosecurity and the Law
133,3.000,MSFS 556,Cybersecurity has emerged as one of the most s...,secur,Fall 2017,Cybersecurity: Bus/Gov/Int Com


In [195]:
georgetown = georgetown[['title', 'dept_num', 'description', 'credits', 'term', 'keyword']]
georgetown

,title,dept_num,description,credits,term,keyword
124,Cybersecurity Seminar,COSC 437,1.000 Credit hours\n1.000 Lecture hours\n\nLev...,1.000,Fall 2017,secur
125,Cybersecurity Writing Seminar,COSC 438,1.000 Credit hours\n1.000 Lecture hours\n\nLev...,1.000,Fall 2017,secur
126,Cybercrime & Tech Insecurity,LAWG 1409,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,2.000,Fall 2017,secur
127,Biosecurity and the Law,LAWG 2004,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,2.000,Fall 2017,secur
128,Cybersecurity Law,LAWG 2052,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,2.000,Fall 2017,secur
129,Biosecurity and the Law,LAWG 3038,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,2.000,Fall 2017,secur
130,Cybercrime & Tech Insecurity,LAWJ 1409,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,2.000,Fall 2017,secur
131,Biosecurity and the Law,LAWJ 2004,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,2.000,Fall 2017,secur
132,Biosecurity and the Law,LAWJ 3038,2.000 Credit hours\n2.000 Lecture hours\n\nLev...,2.000,Fall 2017,secur
133,Cybersecurity: Bus/Gov/Int Com,MSFS 556,Cybersecurity has emerged as one of the most s...,3.000,Fall 2017,secur


Lastly, we want to export our csv. Ideally, all csv files should be written to the courses directory in our repository. 

In [196]:
#export as csv
georgetown.to_csv('../courses/6-Georgetown-University-Catalog.csv', index=False)